In [1]:
%matplotlib ipympl
import os.path as op
from mayavi import mlab
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import randn
from scipy import stats as stats
import mne
from mne.datasets import fetch_fsaverage

## Set method, time window and noise covariance matrix

In [2]:
method = 'MNE'
tmin = 0.125
tmax = 0.220
noise_cov = 'with_reg_noise_cov'

## Create cross-hemisphere contrast and run t-test 

In [3]:
fs_dir = fetch_fsaverage(verbose=False)
subjects_dir = op.dirname(fs_dir)
data_folder = '/home/zsuzsanna/Documents/MTA/source_localization/natural_reading/'
# excluded: 181584
subjects = ['842608', '587631', '217720', '059694', '394107', '356349', '044846', '050120', '269257', '103183', '862169', '284297', '643438', '048298', '414822', '638838', '390744', '930517', '093925', '213103', '331536', '205515', '135230', '320268', '319897', '321319', '303786', '822816', '163753', '667207', '424174', '612370', '528213', '009833', '927179', '515155', '366394', '133288']
print('Number of participants included: ',len(subjects))

## Dummy variables for arrays built in the for loop
stc_data = None
stc_xhemi_data = None
tstep = None

for subject in subjects:
    stc_file = op.join(data_folder, noise_cov, method, subject)
    stc = mne.read_source_estimate(stc_file, 'fsaverage')
    stc.crop(tmin, tmax)
    tstep = stc.tstep
    ## Morph to fsaverage_sym
    stc = mne.compute_source_morph(stc, 'fsaverage', 'fsaverage_sym', smooth=5,
                                   warn=False,
                                   subjects_dir=subjects_dir).apply(stc)
    # Compute a morph-matrix mapping the right to the left hemisphere,
    # and vice-versa.
    morph = mne.compute_source_morph(stc, 'fsaverage_sym', 'fsaverage_sym',
                                    spacing=stc.vertices, warn=False,
                                    subjects_dir=subjects_dir, xhemi=True,
                                    verbose='error')  # creating morph map
    stc_xhemi = morph.apply(stc)
    
    if np.all(stc_data) == None:
        stc_data = stc.data
        stc_xhemi_data = stc_xhemi.data
    else:
        stc_data = np.dstack((stc_data, stc.data))
        stc_xhemi_data = np.dstack((stc_xhemi_data, stc_xhemi.data))
print('Collected {} stc data files'.format(stc_data.shape[2]))
    
X = stc_data[:,:,:] - stc_xhemi_data[:,:,:]
out = stats.ttest_1samp(X, 0, axis=2) ## compute across participants
ts = [out[0]]
ps = [out[1]]

Number of participants included:  38
Collected 38 stc data files


## Visualization with time viewer 

In [ ]:
stc_file = op.join(data_folder, noise_cov, method, '842608')
stc = mne.read_source_estimate(stc_file, 'fsaverage').crop(tmin,tmax)
ps_for_viz = ps[0] * -1
stc.data = ps_for_viz

views = ['lat','caudal','ventral']
mlab.options.offscreen = False
tview_kwargs = dict(hemi='both', clim=dict(kind='value', lims=[-0.06, -0.04,0.0]),
                    views=views, time_unit='s', smoothing_steps=5, colorbar = True, size = 1400, time_viewer = True)
brain = stc.plot(**tview_kwargs)